# Etapa 1

## Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importação de Dados


In [2]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/gabrielramos731/Data-science-projects/main/Air%20Line%20Satisfaction%20Prediction/data/train.csv')


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
df = df_raw.copy()
df = df.drop(['Unnamed: 0', 'id'], axis=1)

In [ ]:
df['Type of Travel'] = df['Type of Travel'].str.replace('Business travel','Work travel')
df.head()

## Documentação dos Dados
Gender: Gender of the passengers (Female, Male)

Customer Type: The customer type (Loyal customer, disloyal customer)

Age: The actual age of the passengers

Type of Travel: Purpose of the flight of the passengers (Personal Travel, Business Travel)

Class: Travel class in the plane of the passengers (Business, Eco, Eco Plus)

Flight distance: The flight distance of this journey

Inflight wifi service: Satisfaction level of the inflight wifi service (0:Not Applicable;1-5)

Departure/Arrival time convenient: Satisfaction level of Departure/Arrival time convenient

Ease of Online booking: Satisfaction level of online booking

Gate location: Satisfaction level of Gate location

Food and drink: Satisfaction level of Food and drink

Online boarding: Satisfaction level of online boarding

Seat comfort: Satisfaction level of Seat comfort

Inflight entertainment: Satisfaction level of inflight entertainment

On-board service: Satisfaction level of On-board service

Leg room service: Satisfaction level of Leg room service

Baggage handling: Satisfaction level of baggage handling

Check-in service: Satisfaction level of Check-in service

Inflight service: Satisfaction level of inflight service

Cleanliness: Satisfaction level of Cleanliness

Departure Delay in Minutes: Minutes delayed when departure

Arrival Delay in Minutes: Minutes delayed when Arrival

Satisfaction: Airline satisfaction level(Satisfaction, neutral or dissatisfaction)

## Análise de Dados Univariável

In [ ]:
df.dtypes

In [ ]:
var_cat = []
var_num = []
for i,j in zip(df.columns, df.dtypes):
  if j == 'object':
    var_cat.append(i)
  else:
    var_num.append(i)

### Valores Duplicados

In [ ]:
df.duplicated().sum()

### Valores Nulos


In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()

### Box-plots

In [ ]:

fig, axes = plt.subplots(3, 6)

for i, el in enumerate(var_num):
  a = df.boxplot(el, ax=axes.flatten()[i], fontsize='large')

fig.set_size_inches(18.5, 8.5)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 6)

for i, el in enumerate(var_num):
  a = df.hist(el, ax=axes.flatten()[i])

fig.set_size_inches(18.5, 8.5)
plt.tight_layout()
plt.show()

### Gráficos de Contagem

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(10, 8))

for i, el in enumerate(var_cat):
  sns.countplot(x=df[el], ax=axes.flatten()[i])

  plt.tight_layout()

## Hipótesis

1.  O quão a classe influencia na satisfação do cliente

2. Qual tipo de viagem impacta mais na satisfação

3. O atraso é a variável de maior impacto na satisfação (primeiro modelo)?

## Análise de dados bivariável

### Influência da classe na satisfação

In [ ]:
df_grp = df.groupby('Class')['satisfaction'].value_counts().to_frame()

ls = []
for x in df['Class'].value_counts().values:
  ls.append(x)
  ls.append(x)

df_grp['proporcao'] = df_grp['satisfaction'] / ls * 100
df_grp.sort_index(ascending=False, inplace=True)
df_grp

### Tipo de viagem e impacto na satisfação

In [ ]:
df_grp = df.groupby('Type of Travel')['satisfaction'].value_counts().to_frame()

ls = []
for x in df.groupby('Type of Travel')['Type of Travel'].value_counts().values:
  ls.append(x)
  ls.append(x)

df_grp['proporcao'] = df_grp['satisfaction'] / ls *100
df_grp.sort_index(ascending=False, inplace=True)
df_grp

**A justificativa para diferença tão grande na satisfação pelo tipo de viagem.**

In [ ]:
df.groupby('Type of Travel')[['Flight Distance',
                              'Leg room service', 
                              'Departure Delay in Minutes',
                              'Departure/Arrival time convenient']].mean()

In [ ]:
df.groupby('Class')[['Flight Distance',
                     'Leg room service', 
                     'Departure Delay in Minutes',
                     'Departure/Arrival time convenient']].mean()

### Impacto de classe e viagem na satisfação

In [ ]:
df_grp2 = df.groupby(['Type of Travel','Class'])['satisfaction'].value_counts().to_frame()

In [ ]:
ls_travels = ['Personal Travel','Work travel']
ls_class = ['Business','Eco', 'Eco Plus']
ls = []

for travel in ls_travels:
    for i_class in ls_class:
        ls.append(df[(df['Type of Travel'] == travel) & (df['Class'] == i_class)].shape[0])
        ls.append(df[(df['Type of Travel'] == travel) & (df['Class'] == i_class)].shape[0])
        
df_grp2['proporcao'] = df_grp2['satisfaction']/ls *100
df_grp2.sort_index(ascending=False, inplace=True)
df_grp2

**É possível ver que em uma mesma classe, viagens a trabalho possuem um índice de satisfação superior comparado a viagens pessoais.**

# Etapa 2

## Tratamento das variáveis categóricas

In [ ]:
!pip install category_encoders

In [ ]:
from category_encoders.one_hot import OneHotEncoder, OrdinalEncoder
from sklearn import preprocessing

## Map para Class

In [ ]:
df_teste = df.copy()
df_teste['Class'] = df_teste['Class'].map({'Business':3, 'Eco Plus':2, 'Eco':1})
df_teste.head(10)

####Rótulos

1: Eco

2: Eco Plus

3: Business

## Label Encoder

In [ ]:
df_lb_enc = df_teste.copy()
colunas = ['Gender', 'Customer Type', 'Type of Travel', 'satisfaction']
le = preprocessing.LabelEncoder()
for i in colunas:
  df_lb_enc[i] = le.fit_transform(df_teste[i])
df_encod = df_lb_enc.copy()
df_encod.head()

In [ ]:
df_encod.info()

## Correlações


In [ ]:
plt.figure(figsize=(15,15));
sns.heatmap(df_encod.corr(), annot=True, fmt='.2f', cmap="crest");

## Modelo de Classificação

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

### Árvore de Decisão

In [ ]:
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score

In [ ]:
def treinamento(meu_modelo, minha_max_depth=3):
  X_train, X_test, y_train, y_test = train_test_split(df_encod[modelo],
                                                      df_encod['satisfaction'],
                                                      test_size=0.25, 
                                                      random_state=42)
  classifier = DecisionTreeClassifier(random_state=42, max_depth=minha_max_depth)
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)

  acuracia = classifier.score(X_test, y_test)

  return classifier, acuracia, y_pred, y_test

In [ ]:
modelo = ['Type of Travel', 'Class']

classifier, acuracia, y_pred, y_test = treinamento(modelo)
print(acuracia)

#### Plot Árvore de Decisão

In [ ]:
plt.figure(figsize=(14,8), dpi=100)
plot_tree(classifier, feature_names=classifier.feature_names_in_, fontsize=9);

### Validação Cruzada

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold

In [ ]:
modelo = ['Type of Travel', 'Class']
X = df_encod[modelo]
y = df_encod['satisfaction']

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

values = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy')
print(f'media: {values.mean()}\nmin: {min(values)}\nmax: {max(values)}')

### Matriz de Confusão

In [ ]:
conf_mat_classifier = confusion_matrix(y_test, y_pred)
a = sns.heatmap(conf_mat_classifier, annot=True, 
                cmap='Blues', fmt='.0f', 
                xticklabels=['Não satisfeito', 'Satisfeito'],
                yticklabels=['Não satisfeito', 'Satisfeito'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de confusão');

### Teste da Hipótese - Variáveis Sozinhas

#### Apenas `Type of Travel`

In [ ]:
modelo = ['Type of Travel']

classifier, _, y_pred, y_test = treinamento(modelo, 30)

X = df_encod[modelo]
y = df_encod['satisfaction']

cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)

values = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy')
print(f'media: {values.mean()}\nmin: {min(values)}\nmax: {max(values)}')

In [ ]:
plt.figure(figsize=(8,8), dpi=100)
plot_tree(classifier, feature_names=classifier.feature_names_in_);

In [ ]:
conf_mat_classifier = confusion_matrix(y_test, y_pred)
a = sns.heatmap(conf_mat_classifier, annot=True, 
                cmap='Blues', fmt='.0f', 
                xticklabels=['Não satisfeito', 'Satisfeito'],
                yticklabels=['Não satisfeito', 'Satisfeito'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de confusão');

#### Apenas `Class`

In [ ]:
modelo = ['Class']

classifier, _, y_pred, y_test = treinamento(modelo)

X = df_encod[modelo]
y = df_encod['satisfaction']

cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)

values = cross_val_score(classifier, X, y, cv=cv, scoring='accuracy')
print(f'media: {values.mean()}\nmin: {min(values)}\nmax: {max(values)}')

In [ ]:
plt.figure(figsize=(8,8), dpi=100)
plot_tree(classifier, feature_names=classifier.feature_names_in_);

In [ ]:
conf_mat_classifier = confusion_matrix(y_test, y_pred)
a = sns.heatmap(conf_mat_classifier, annot=True, 
                cmap='Blues', fmt='.0f', 
                xticklabels=['Não satisfeito', 'Satisfeito'],
                yticklabels=['Não satisfeito', 'Satisfeito'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de confusão');

# Etapa 3

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score

# SEED = 42
# np.random.seed(SEED)

# cv = StratifiedKFold(n_splits = 10, shuffle = True)
# knn = KNeighborsClassifier(n_neighbors=3)
# model = DecisionTreeClassifier(max_depth=3)
# model_log = LogisticRegression(solver='liblinear')
# model_rand = RandomForestClassifier(n_estimators=100)

# models = [knn, model, model_log, model_rand]
# name = ['KNN','Árvore de Decisão', 
#         'Regressão Logística', 'Random Forest']
# count = 0
# for item in models:
#     np.random.seed(SEED)
#     results = cross_val_score(item, X, y, cv = cv, scoring = 'accuracy')
#     mean = results.mean()
#     dv = results.std()
    
#     print('Acurácia média - Modelo {}: {:.2f}%'.format(name[count], mean*100))
#     print('Intervalo de acurácia - Modelo {}: [{:.2f}% ~ {:.2f}%]\n'.format(name[count], (mean - 2*dv)*100, (mean + 2*dv)*100))
#     count += 1